# 🎯 Day 1 실습 4: 베이스라인 vs 파인튜닝 모델 성능 비교

## 학습 목표
- **핵심 목표**: 파인튜닝 전 (베이스라인) vs 후 (파인튜닝) 모델 성능을 정량적으로 비교
- ROUGE, BLEU, 코사인 유사도 등 다양한 메트릭으로 성능 측정
- RAG 시나리오에서의 실제 성능 개선 확인
- 시각화를 통한 성능 차이 분석
- 파인튜닝 효과에 대한 종합적 평가

### 💡 Day 1 실습의 핵심
**베이스라인(원본 모델) vs 파인튜닝 모델** 성능 비교를 통해 
**RAFT 파인튜닝이 실제로 RAG 성능을 개선했는지 검증**하는 것이 이 실습의 가장 중요한 목표입니다!

In [ ]:
# 필요한 라이브러리 확인 및 설치 (이미 설치된 경우 스킵)
import importlib
import subprocess
import sys

def check_and_install_package(package_name, import_name=None, version=None):
    """
    패키지 존재 여부 확인 후 필요시에만 설치
    
    Args:
        package_name: pip로 설치할 패키지명
        import_name: import할 모듈명 (None이면 package_name 사용)
        version: 특정 버전 지정 (None이면 버전 체크 안함)
    """
    if import_name is None:
        import_name = package_name.replace('-', '_')
    
    try:
        # 패키지가 이미 설치되어 있는지 확인
        module = importlib.import_module(import_name)
        
        if version is not None and hasattr(module, '__version__'):
            current_version = module.__version__
            print(f"✅ {package_name} 이미 설치됨 (버전: {current_version})")
        else:
            print(f"✅ {package_name} 이미 설치됨")
        
        return True
        
    except ImportError:
        print(f"📦 {package_name} 설치 중...")
        try:
            if version:
                subprocess.run([sys.executable, "-m", "pip", "install", "-q", f"{package_name}=={version}"], 
                             check=True, capture_output=True)
            else:
                subprocess.run([sys.executable, "-m", "pip", "install", "-q", package_name], 
                             check=True, capture_output=True)
            print(f"✅ {package_name} 설치 완료")
            return True
        except subprocess.CalledProcessError as e:
            print(f"❌ {package_name} 설치 실패: {e}")
            return False

print("🔍 라이브러리 의존성 확인 중...")

# 필요한 패키지들 확인 및 설치
packages_to_check = [
    # 코어 라이브러리들은 이미 03번에서 설치했으므로 체크만
    ("transformers", "transformers"),
    ("torch", "torch"),
    ("peft", "peft"),
    
    # 평가 전용 라이브러리들만 필요시 설치
    ("rouge-score", "rouge_score"),
    ("nltk", "nltk"),
    ("scikit-learn", "sklearn"),
    
    # 시각화 라이브러리들
    ("matplotlib", "matplotlib"),
    ("seaborn", "seaborn"),
    
    # 데이터 처리
    ("pandas", "pandas"),
    ("numpy", "numpy"),
    ("tqdm", "tqdm")
]

print("📋 패키지 확인 결과:")
for package_name, import_name in packages_to_check:
    check_and_install_package(package_name, import_name)

print("\n🎉 모든 의존성 확인 완료!")
print("💡 이미 설치된 패키지들은 재설치하지 않았습니다.")

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    pipeline, BitsAndBytesConfig
)
from peft import PeftModel
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from datasets import load_dataset
from tqdm import tqdm
import json
import os
import glob
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib) - 모델 성능 비교 차트에서 한글이 깨지지 않도록 설정
print("🔧 한글 폰트 설정 중...")
!apt-get update -qq
!apt-get install fonts-nanum -qq > /dev/null

import matplotlib.font_manager as fm

# 나눔바른고딕 폰트 경로 설정
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
fm.fontManager.addfont(fontpath)

# matplotlib 설정 업데이트 - 모든 평가 결과 차트에서 한글이 정상적으로 표시됨
plt.rcParams.update({
    'font.family': 'NanumBarunGothic',  # 기본 폰트를 나눔바른고딕으로 설정
    'axes.unicode_minus': False         # 음수 기호 표시 문제 해결 (성능 차이 표시에서 중요)
})

# 시각화 스타일 설정 - 더 전문적이고 깔끔한 성능 비교 차트를 위한 설정
plt.style.use('default')              # 기본 스타일 사용
sns.set_palette("Set2")               # 구별하기 쉬운 색상 팔레트 설정

print("✅ 한글 폰트 설정 완료 - 성능 평가 차트에서 한글이 정상 표시됩니다")
print("📦 라이브러리 import 완료!")

In [ ]:
# 모델 및 평가 환경 설정
MODEL_NAME = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 파인튜닝된 모델 경로 자동 탐지 - 개선된 버전
def find_fine_tuned_model_path():
    """파인튜닝된 모델 경로를 자동으로 찾는 함수 - 개선된 탐지 알고리즘"""
    print("🔍 파인튜닝된 모델 검색 중...")
    
    # 1. 다양한 경로 패턴으로 검색
    possible_patterns = [
        "./fine_tuned_model*",     # 기본 타임스탬프 패턴
        "./exaone_raft_lora_*",    # 03번 노트북에서 생성하는 새 패턴
        "./outputs/*",             # 대체 경로
        "./*lora*",               # LoRA가 포함된 모든 디렉토리
        "./*fine*",               # fine이 포함된 모든 디렉토리
        "./*exaone*",             # exaone이 포함된 모든 디렉토리
    ]
    
    found_models = []
    
    for pattern in possible_patterns:
        matches = glob.glob(pattern)
        for match in matches:
            if os.path.isdir(match):  # 디렉토리인지 확인
                # 필수 파일들이 존재하는지 확인
                adapter_config = os.path.join(match, "adapter_config.json")
                adapter_model = os.path.join(match, "adapter_model.safetensors")
                
                if os.path.exists(adapter_config) and os.path.exists(adapter_model):
                    # 파일 크기 확인 (빈 파일이 아닌지)
                    if os.path.getsize(adapter_config) > 0 and os.path.getsize(adapter_model) > 0:
                        modification_time = os.path.getctime(match)
                        found_models.append({
                            "path": match,
                            "mtime": modification_time,
                            "config_size": os.path.getsize(adapter_config),
                            "model_size": os.path.getsize(adapter_model)
                        })
                        print(f"  ✅ 발견: {match}")
                        print(f"      Config: {os.path.getsize(adapter_config):,} bytes")
                        print(f"      Model:  {os.path.getsize(adapter_model):,} bytes")
    
    if not found_models:
        print("  ❌ 유효한 파인튜닝 모델을 찾을 수 없음")
        return None
    
    # 2. 가장 최근 모델 선택
    latest_model = max(found_models, key=lambda x: x["mtime"])
    selected_path = latest_model["path"]
    
    print(f"\n🎯 선택된 모델: {selected_path}")
    print(f"   생성 시간: {datetime.fromtimestamp(latest_model['mtime'])}")
    print(f"   모델 크기: {latest_model['model_size']:,} bytes")
    
    # 3. 추가 유효성 검사
    try:
        # adapter_config.json 파일을 읽어서 올바른 형식인지 확인
        with open(os.path.join(selected_path, "adapter_config.json"), 'r') as f:
            config = json.load(f)
        
        if "peft_type" in config and config.get("peft_type") == "LORA":
            print(f"   ✅ LoRA 설정 확인됨")
            return selected_path
        else:
            print(f"   ⚠️ LoRA 설정이 올바르지 않음")
            return None
            
    except Exception as e:
        print(f"   ❌ 설정 파일 검증 실패: {e}")
        return None

def check_03_notebook_completion():
    """
    03번 노트북의 실행 상태를 확인하는 함수
    """
    print("\n📋 03번 노트북 실행 상태 확인:")
    
    # 1. 전처리된 데이터 확인
    data_files = [
        "processed_data/train_raft_ko.jsonl",
        "processed_data/valid_raft_ko.jsonl",
        "processed_data/metadata.json"
    ]
    
    data_ready = True
    for file_path in data_files:
        if os.path.exists(file_path):
            file_size = os.path.getsize(file_path)
            print(f"  ✅ {file_path}: {file_size:,} bytes")
        else:
            print(f"  ❌ {file_path}: 파일 없음")
            data_ready = False
    
    # 2. 파인튜닝 모델 확인
    model_path = find_fine_tuned_model_path()
    
    # 3. 결과 요약
    if not data_ready:
        print(f"\n⚠️ 전처리 데이터가 없습니다.")
        print(f"   👉 01_data_preprocessing_and_validation.ipynb를 먼저 실행하세요.")
    
    if not model_path:
        print(f"\n⚠️ 파인튜닝 모델이 없습니다.")
        print(f"   👉 03_fine_tuning_with_lora.ipynb를 먼저 실행하세요.")
        print(f"\n💡 03번 노트북 실행 순서:")
        print(f"   1. 모든 셀을 순서대로 실행")
        print(f"   2. 19번 셀(학습 실행)에서 완료될 때까지 대기")
        print(f"   3. 23번 셀(모델 저장)에서 저장 완료 확인")
        return False
    
    print(f"\n✅ 03번 노트북이 정상적으로 완료되었습니다.")
    return True

# 파인튜닝 모델 경로 찾기
ADAPTER_PATH = find_fine_tuned_model_path()

print(f"🔧 평가 환경 설정:")
print(f"  디바이스: {device}")
print(f"  베이스 모델: {MODEL_NAME}")

if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"  GPU 메모리: {gpu_memory:.1f}GB")
else:
    print("  ⚠️ CUDA를 사용할 수 없습니다. CPU에서 실행됩니다.")

# 실행 상태 종합 확인
notebook_ready = check_03_notebook_completion()

if ADAPTER_PATH and notebook_ready:
    print(f"\n🎉 완전한 베이스라인 vs 파인튜닝 모델 비교 준비 완료!")
    print(f"  파인튜닝 모델: {ADAPTER_PATH}")
elif ADAPTER_PATH:
    print(f"\n✅ 파인튜닝 모델 발견: {ADAPTER_PATH}")
    print(f"🔥 베이스라인 vs 파인튜닝 모델 비교 가능!")
else:
    print(f"\n⚠️ 파인튜닝 모델을 찾을 수 없습니다!")
    print(f"🔄 현재는 베이스라인 모델만 평가됩니다.")
    print(f"\n💡 파인튜닝 모델을 생성하려면:")
    print(f"   1. 03_fine_tuning_with_lora.ipynb 열기")
    print(f"   2. 모든 셀을 처음부터 끝까지 실행")
    print(f"   3. 학습 완료 후 모델 저장 확인")
    print(f"   4. 이 노트북을 다시 실행")

## 1. 평가 환경 설정

## 2. 평가 메트릭 구현

In [ ]:
class EvaluationMetrics:
    """다양한 평가 메트릭을 계산하는 클래스"""
    
    def __init__(self):
        self.rouge_scorer = rouge_scorer.RougeScorer(
            ['rouge1', 'rouge2', 'rougeL'], use_stemmer=True
        )
        self.smoothing = SmoothingFunction().method4
        self.tfidf = TfidfVectorizer()
    
    def compute_rouge(self, predictions, references):
        """ROUGE 점수 계산"""
        rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
        
        for pred, ref in zip(predictions, references):
            scores = self.rouge_scorer.score(ref, pred)
            for key in rouge_scores:
                rouge_scores[key].append(scores[key].fmeasure)
        
        return {key: np.mean(values) for key, values in rouge_scores.items()}
    
    def compute_bleu(self, predictions, references):
        """BLEU 점수 계산"""
        bleu_scores = []
        
        for pred, ref in zip(predictions, references):
            pred_tokens = pred.split()
            ref_tokens = [ref.split()]  # BLEU는 reference를 리스트로 요구
            
            try:
                score = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=self.smoothing)
                bleu_scores.append(score)
            except:
                bleu_scores.append(0.0)
        
        return np.mean(bleu_scores)
    
    def compute_cosine_similarity(self, predictions, references):
        """코사인 유사도 계산"""
        try:
            all_texts = predictions + references
            tfidf_matrix = self.tfidf.fit_transform(all_texts)
            
            pred_vectors = tfidf_matrix[:len(predictions)]
            ref_vectors = tfidf_matrix[len(predictions):]
            
            similarities = []
            for i in range(len(predictions)):
                sim = cosine_similarity(
                    pred_vectors[i:i+1], ref_vectors[i:i+1]
                )[0][0]
                similarities.append(sim)
            
            return np.mean(similarities)
        except:
            return 0.0

evaluator = EvaluationMetrics()
print("✅ 평가 메트릭 클래스 초기화 완료")

## 3. 평가 데이터셋 준비

In [ ]:
def load_evaluation_dataset():
    """
    평가 데이터셋 로드 함수 - RAG 성능 평가에 적합한 데이터를 준비
    """
    print("🔄 평가 데이터셋 로드 중...")
    
    try:
        # 전처리된 데이터가 있는지 먼저 확인
        if os.path.exists("processed_data/valid_raft_ko.jsonl"):
            print("📁 전처리된 검증 데이터 사용")
            import jsonlines
            
            valid_data = []
            with jsonlines.open("processed_data/valid_raft_ko.jsonl", "r") as reader:
                valid_data = list(reader)
            
            # RAFT 형식에서 평가 형식으로 변환
            eval_samples = []
            for item in valid_data[:50]:  # 평가 시간 고려하여 50개 제한
                if "original_question" in item and "original_answer" in item:
                    # Context 추출 (User 메시지에서)
                    user_content = item["messages"][1]["content"] if "messages" in item else ""
                    context_match = user_content.split("=== 질문 ===")[0] if "=== 질문 ===" in user_content else ""
                    context = context_match.replace("=== 컨텍스트 ===", "").strip()
                    
                    eval_samples.append({
                        "context": context,
                        "question": item["original_question"],
                        "answer": item["original_answer"]
                    })
            
            print(f"✅ 전처리된 검증 데이터에서 {len(eval_samples)}개 샘플 로드")
            return eval_samples
            
    except Exception as e:
        print(f"⚠️ 전처리된 데이터 로드 실패: {e}")
    
    # 대안: 원본 데이터셋 사용
    try:
        print("📂 원본 데이터셋 로드")
        dataset = load_dataset("neural-bridge/rag-dataset-12000")
        
        eval_size = min(50, len(dataset["train"]))  # 평가 시간 단축
        eval_data = dataset["train"].select(range(eval_size))
        
        eval_samples = []
        for item in eval_data:
            eval_samples.append({
                "context": item.get("context", ""),
                "question": item.get("question", ""),
                "answer": item.get("answer", "")
            })
        
        print(f"✅ 원본 데이터셋에서 {len(eval_samples)}개 샘플 로드")
        return eval_samples
        
    except Exception as e:
        print(f"❌ 데이터셋 로드 실패: {e}")
        
        # 최종 대안: 샘플 데이터 생성
        print("🔄 샘플 평가 데이터 생성")
        sample_data = [
            {
                "context": "한국의 수도는 서울입니다. 서울은 한강을 중심으로 발달했으며, 약 천만 명의 인구가 거주합니다.",
                "question": "한국의 수도는 어디인가요?",
                "answer": "한국의 수도는 서울입니다."
            },
            {
                "context": "김치는 한국의 전통 발효 식품입니다. 배추와 다양한 양념을 사용하여 만들며, 건강에 좋은 유산균이 풍부합니다.",
                "question": "김치는 어떤 음식인가요?",
                "answer": "김치는 한국의 전통 발효 식품으로, 배추와 양념으로 만들어지며 유산균이 풍부합니다."
            },
            {
                "context": "인공지능은 컴퓨터 시스템이 인간의 지능적인 행동을 모방할 수 있도록 하는 기술입니다. 머신러닝과 딥러닝이 핵심 기술입니다.",
                "question": "인공지능이란 무엇인가요?",
                "answer": "인공지능은 컴퓨터가 인간의 지능적 행동을 모방하는 기술로, 머신러닝과 딥러닝이 핵심입니다."
            }
        ]
        
        print(f"📝 {len(sample_data)}개 샘플 데이터 생성 완료")
        return sample_data

# 평가 데이터 로드
eval_samples = load_evaluation_dataset()
print(f"\n📊 평가 데이터 준비 완료: {len(eval_samples)}개 샘플")
if eval_samples:
    print(f"\n📋 첫 번째 샘플 미리보기:")
    print(f"  Context: {eval_samples[0]['context'][:100]}...")
    print(f"  Question: {eval_samples[0]['question']}")
    print(f"  Answer: {eval_samples[0]['answer']}")

## 4. 모델 로드

In [ ]:
def load_models():
    """
    베이스라인과 파인튜닝 모델을 모두 로드하는 함수
    """
    print("🔄 모델 로드 중...")
    
    # 4-bit 양자화 설정 (메모리 효율성)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,  # 안정성을 위해 bfloat16 사용
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    
    # 토크나이저 로드
    print("📝 토크나이저 로드...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    print(f"✅ 토크나이저 로드 완료 (vocab size: {tokenizer.vocab_size:,})")
    
    # 베이스라인 모델 로드
    print("🤖 베이스라인 모델 로드...")
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True
    )
    print("✅ 베이스라인 모델 로드 완료")
    
    # 파인튜닝 모델 로드 시도 - 개선된 오류 처리
    fine_tuned_model = None
    if ADAPTER_PATH and os.path.exists(ADAPTER_PATH):
        try:
            print("🎯 파인튜닝 모델 로드...")
            print(f"   모델 경로: {ADAPTER_PATH}")
            
            # 필수 파일 존재 확인
            required_files = [
                "adapter_config.json",
                "adapter_model.safetensors"
            ]
            
            missing_files = []
            for file_name in required_files:
                file_path = os.path.join(ADAPTER_PATH, file_name)
                if not os.path.exists(file_path):
                    missing_files.append(file_name)
                else:
                    file_size = os.path.getsize(file_path)
                    print(f"   ✅ {file_name}: {file_size:,} bytes")
            
            if missing_files:
                print(f"   ❌ 필수 파일이 없습니다: {', '.join(missing_files)}")
                raise FileNotFoundError(f"Missing required files: {missing_files}")
            
            # PEFT 어댑터 적용
            fine_tuned_model = PeftModel.from_pretrained(
                base_model, 
                ADAPTER_PATH,
                torch_dtype=torch.bfloat16
            )
            print("✅ 파인튜닝 모델 로드 완료")
            
        except Exception as e:
            error_msg = str(e)
            print(f"❌ 파인튜닝 모델 로드 실패: {error_msg}")
            
            # 구체적인 오류 분석 및 해결 방안 제시
            if "adapter_config.json" in error_msg:
                print(f"\n🔍 문제 분석: LoRA 어댑터 설정 파일을 찾을 수 없습니다.")
                print(f"💡 해결 방법:")
                print(f"   1. 03번 노트북에서 파인튜닝이 완료되었는지 확인")
                print(f"   2. 23번 셀(모델 저장)이 성공적으로 실행되었는지 확인")
                print(f"   3. 저장된 모델 경로에 다음 파일들이 있는지 확인:")
                for file_name in required_files:
                    file_path = os.path.join(ADAPTER_PATH, file_name)
                    if os.path.exists(file_path):
                        print(f"      ✅ {file_name}")
                    else:
                        print(f"      ❌ {file_name} (없음)")
                        
            elif "safetensors" in error_msg or "model" in error_msg:
                print(f"\n🔍 문제 분석: LoRA 어댑터 모델 파일에 문제가 있습니다.")
                print(f"💡 해결 방법:")
                print(f"   1. 03번 노트북의 학습이 중단 없이 완료되었는지 확인")
                print(f"   2. GPU 메모리 부족으로 저장이 실패했을 가능성 확인")
                print(f"   3. 03번 노트북을 처음부터 다시 실행")
                
            else:
                print(f"\n🔍 문제 분석: 일반적인 모델 로드 오류")
                print(f"💡 해결 방법:")
                print(f"   1. 03번 노트북 전체를 다시 실행")
                print(f"   2. GPU 메모리 부족 시 런타임 재시작 후 재실행")
                print(f"   3. 파인튜닝 설정을 더 보수적으로 조정")
            
            print(f"\n📋 현재 상태:")
            print(f"   베이스라인 모델: ✅ 로드됨")
            print(f"   파인튜닝 모델: ❌ 로드 실패")
            print(f"   평가 가능 여부: 베이스라인만 평가 가능")
            print(f"💡 베이스라인 모델만으로 평가를 진행합니다.")
            fine_tuned_model = None
            
    else:
        print("⚠️ 파인튜닝 모델 경로를 찾을 수 없음")
        print(f"\n📋 파인튜닝 모델 생성 가이드:")
        print(f"   1. 03_fine_tuning_with_lora.ipynb 노트북 열기")
        print(f"   2. 셀 1-2: 라이브러리 설치")
        print(f"   3. 셀 3-11: 데이터 로드 및 전처리")
        print(f"   4. 셀 12-19: 모델 설정 및 학습 실행 (시간이 오래 걸림)")
        print(f"   5. 셀 20-23: 결과 분석 및 모델 저장")
        print(f"   6. 이 노트북으로 돌아와서 다시 실행")
    
    return tokenizer, base_model, fine_tuned_model

# 파인튜닝 모델 상태 진단 함수
def diagnose_fine_tuning_status():
    """
    파인튜닝 상태를 진단하고 사용자에게 구체적인 가이드 제공
    """
    print("🔍 파인튜닝 상태 진단:")
    
    # 1. 데이터 준비 상태 확인
    data_status = "✅"
    if not os.path.exists("processed_data"):
        data_status = "❌"
        print("   데이터 전처리: ❌ (01번 노트북 미실행)")
    elif not os.path.exists("processed_data/train_raft_ko.jsonl"):
        data_status = "⚠️"
        print("   데이터 전처리: ⚠️ (일부 파일 누락)")
    else:
        print("   데이터 전처리: ✅")
    
    # 2. 파인튜닝 모델 상태 확인
    if ADAPTER_PATH:
        print(f"   파인튜닝 모델: ✅ 경로 발견 ({ADAPTER_PATH})")
        
        # 파일 상세 확인
        config_path = os.path.join(ADAPTER_PATH, "adapter_config.json")
        model_path = os.path.join(ADAPTER_PATH, "adapter_model.safetensors")
        
        if os.path.exists(config_path) and os.path.exists(model_path):
            config_size = os.path.getsize(config_path)
            model_size = os.path.getsize(model_path)
            print(f"      Config 파일: {config_size:,} bytes")
            print(f"      Model 파일: {model_size:,} bytes")
            
            if model_size < 1000000:  # 1MB보다 작으면 문제
                print("      ⚠️ 모델 파일이 너무 작습니다. 학습이 제대로 되지 않았을 수 있습니다.")
            else:
                print("      ✅ 파일 크기가 정상적입니다.")
        else:
            print("   파인튜닝 모델: ❌ 필수 파일 누락")
    else:
        print("   파인튜닝 모델: ❌ 모델 없음")
    
    # 3. 권장 사항 제시
    print(f"\n💡 다음 단계:")
    if data_status == "❌":
        print("   👉 01_data_preprocessing_and_validation.ipynb 먼저 실행")
    elif not ADAPTER_PATH:
        print("   👉 03_fine_tuning_with_lora.ipynb 실행하여 파인튜닝 수행")
        print("      - 예상 소요 시간: 30-60분 (GPU 성능에 따라)")
        print("      - 완료 후 이 노트북 다시 실행")
    else:
        print("   👉 베이스라인 vs 파인튜닝 모델 비교 평가 진행 가능!")

# 모델들 로드
tokenizer, base_model, fine_tuned_model = load_models()

# 파인튜닝 상태 진단 (파인튜닝 모델이 없는 경우)
if fine_tuned_model is None:
    diagnose_fine_tuning_status()

# GPU 메모리 사용량 확인
if torch.cuda.is_available():
    print(f"\n💾 현재 GPU 메모리 사용량:")
    for i in range(torch.cuda.device_count()):
        allocated = torch.cuda.memory_allocated(i) / 1024**3
        total = torch.cuda.get_device_properties(i).total_memory / 1024**3
        print(f"  GPU {i}: {allocated:.1f}GB / {total:.1f}GB ({allocated/total:.1%})")

print(f"\n🎯 로드 완료된 모델:")
print(f"  ✅ 베이스라인 모델: {MODEL_NAME}")
if fine_tuned_model is not None:
    print(f"  ✅ 파인튜닝 모델: {ADAPTER_PATH}")
    print(f"  🔥 베이스라인 vs 파인튜닝 비교 가능!")
else:
    print(f"  ⚠️ 파인튜닝 모델: 로드 실패")
    print(f"  📊 베이스라인 모델만 평가")
    print(f"\n🎯 완전한 비교를 위해서는:")
    print(f"     03_fine_tuning_with_lora.ipynb 완료 → 이 노트북 재실행")

## 5. 모델 추론 및 평가

In [ ]:
def generate_response(model, tokenizer, context, question, max_new_tokens=150):
    """
    모델로부터 응답을 생성하는 함수 - EXAONE 채팅 템플릿 사용
    
    Args:
        model: 추론에 사용할 모델
        tokenizer: 토크나이저
        context: 참조할 컨텍스트
        question: 질문
        max_new_tokens: 생성할 최대 토큰 수
        
    Returns:
        생성된 응답 텍스트
    """
    
    # EXAONE 모델에 최적화된 프롬프트 형식
    messages = [
        {
            "role": "system",
            "content": "당신은 주어진 컨텍스트를 바탕으로 질문에 정확하고 도움이 되는 답변을 제공하는 AI 어시스턴트입니다. 컨텍스트에 없는 정보는 추측하지 말고, 주어진 정보만을 바탕으로 답변하세요."
        },
        {
            "role": "user",
            "content": f"""다음 컨텍스트를 참고하여 질문에 답변해주세요.

=== 컨텍스트 ===
{context}

=== 질문 ===
{question}

위 컨텍스트를 바탕으로 질문에 대한 정확한 답변을 해주세요."""
        }
    ]
    
    # 채팅 템플릿 적용
    try:
        formatted_input = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
    except Exception:
        # 채팅 템플릿이 지원되지 않는 경우 대안 형식
        formatted_input = f"""시스템: 주어진 컨텍스트를 바탕으로 질문에 답변하세요.

사용자: 컨텍스트: {context}

질문: {question}

어시스턴트:"""
    
    # 토크나이징
    inputs = tokenizer(
        formatted_input,
        return_tensors="pt",
        truncation=True,
        max_length=2048  # 컨텍스트가 길어질 수 있으므로 충분한 길이 확보
    )
    
    # GPU 사용 가능시 GPU로 이동
    if torch.cuda.is_available() and hasattr(model, 'device'):
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # 응답 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,                    # 적당한 창의성
            do_sample=True,                     # 샘플링 활성화
            top_p=0.9,                         # Nucleus sampling
            repetition_penalty=1.1,            # 반복 방지
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # 생성된 부분만 디코딩 (입력 제외)
    generated_tokens = outputs[0][inputs['input_ids'].shape[-1]:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    
    # 불필요한 접두어 제거
    if response.startswith("어시스턴트:"):
        response = response[5:].strip()
    elif response.startswith("답변:"):
        response = response[3:].strip()
    
    return response

print("✅ 응답 생성 함수 정의 완료 (EXAONE 최적화)")
print("  - EXAONE 채팅 템플릿 사용")
print("  - 컨텍스트 기반 답변 생성")
print("  - 반복 방지 및 품질 최적화")

In [ ]:
def evaluate_model(model, model_name, eval_samples):
    """
    모델 평가 함수 - RAG 성능을 종합적으로 평가
    
    Args:
        model: 평가할 모델
        model_name: 모델 이름 (결과 표시용)
        eval_samples: 평가 데이터 (context, question, answer 포함)
        
    Returns:
        평가 결과 딕셔너리
    """
    
    print(f"\n🔍 {model_name} 평가 시작...")
    print(f"  평가 샘플 수: {len(eval_samples)}개")
    
    predictions = []
    references = []
    contexts = []
    questions = []
    generation_times = []  # 응답 생성 시간도 측정
    
    # 추론 실행
    for i, sample in enumerate(tqdm(eval_samples, desc=f"{model_name} 추론", ncols=80)):
        context = sample.get('context', '')
        question = sample.get('question', '')
        answer = sample.get('answer', '')
        
        if not context or not question or not answer:
            print(f"  ⚠️ 샘플 {i}: 불완전한 데이터 건너뛰기")
            continue
        
        try:
            # 응답 생성 시간 측정
            import time
            start_time = time.time()
            
            prediction = generate_response(model, tokenizer, context, question)
            
            end_time = time.time()
            generation_time = end_time - start_time
            
            # 결과 저장
            if prediction and prediction.strip():  # 빈 응답 필터링
                predictions.append(prediction.strip())
                references.append(answer.strip())
                contexts.append(context)
                questions.append(question)
                generation_times.append(generation_time)
            else:
                print(f"  ⚠️ 샘플 {i}: 빈 응답 생성")
                
        except Exception as e:
            print(f"  ❌ 샘플 {i} 처리 실패: {str(e)[:100]}...")
            continue
    
    print(f"\n📊 {model_name} 메트릭 계산 중... (유효 샘플: {len(predictions)}개)")
    
    if not predictions:
        print(f"❌ {model_name}: 평가할 수 있는 예측 결과가 없습니다.")
        return None
    
    # 메트릭 계산
    try:
        rouge_scores = evaluator.compute_rouge(predictions, references)
        bleu_score = evaluator.compute_bleu(predictions, references)
        cosine_sim = evaluator.compute_cosine_similarity(predictions, references)
        
        # 추가 메트릭: 평균 응답 길이, 응답 시간
        avg_prediction_length = np.mean([len(pred.split()) for pred in predictions])
        avg_reference_length = np.mean([len(ref.split()) for ref in references])
        avg_generation_time = np.mean(generation_times) if generation_times else 0
        
    except Exception as e:
        print(f"❌ 메트릭 계산 실패: {e}")
        return None
    
    results = {
        'model_name': model_name,
        'sample_count': len(predictions),
        'rouge1': rouge_scores['rouge1'],
        'rouge2': rouge_scores['rouge2'],
        'rougeL': rouge_scores['rougeL'],
        'bleu': bleu_score,
        'cosine_similarity': cosine_sim,
        'avg_prediction_length': avg_prediction_length,
        'avg_reference_length': avg_reference_length,
        'avg_generation_time': avg_generation_time,
        'predictions': predictions,
        'references': references,
        'contexts': contexts,
        'questions': questions
    }
    
    print(f"✅ {model_name} 평가 완료")
    print(f"  ROUGE-1: {rouge_scores['rouge1']:.3f}")
    print(f"  ROUGE-L: {rouge_scores['rougeL']:.3f}")
    print(f"  BLEU: {bleu_score:.3f}")
    print(f"  코사인 유사도: {cosine_sim:.3f}")
    print(f"  평균 응답 시간: {avg_generation_time:.2f}초")
    
    return results

# 모델 평가 실행
if eval_samples:
    print("🎯 베이스라인 vs 파인튜닝 모델 성능 비교 시작!")
    print("=" * 60)
    
    # 베이스라인 모델 평가
    baseline_results = evaluate_model(base_model, "🤖 베이스라인 모델", eval_samples)
    
    # 파인튜닝 모델 평가 (있는 경우)
    finetuned_results = None
    if fine_tuned_model is not None:
        finetuned_results = evaluate_model(fine_tuned_model, "🎯 파인튜닝 모델", eval_samples)
    
    print("\n" + "=" * 60)
    print("✅ 모델 평가 완료!")
    
    if baseline_results and finetuned_results:
        print("🔥 베이스라인 vs 파인튜닝 비교 가능!")
    elif baseline_results:
        print("📊 베이스라인 모델 결과만 사용 가능")
    else:
        print("❌ 평가할 수 있는 결과가 없습니다.")
        
else:
    print("❌ 평가 데이터가 없어 모델 평가를 수행할 수 없습니다.")
    baseline_results = None
    finetuned_results = None

## 6. 평가 결과 시각화

In [ ]:
def create_comparison_table(baseline_results, finetuned_results=None):
    """
    베이스라인과 파인튜닝 모델의 성능 비교 테이블 생성
    
    Args:
        baseline_results: 베이스라인 모델 평가 결과
        finetuned_results: 파인튜닝 모델 평가 결과 (선택사항)
        
    Returns:
        비교 결과 DataFrame
    """
    
    metrics = ['rouge1', 'rouge2', 'rougeL', 'bleu', 'cosine_similarity', 'avg_generation_time']
    metric_names = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BLEU', '코사인 유사도', '응답시간(초)']
    
    data = []
    
    if baseline_results:
        baseline_row = [baseline_results['model_name']] + [baseline_results[metric] for metric in metrics]
        data.append(baseline_row)
    
    if finetuned_results:
        finetuned_row = [finetuned_results['model_name']] + [finetuned_results[metric] for metric in metrics]
        data.append(finetuned_row)
    
    columns = ['모델'] + metric_names
    df = pd.DataFrame(data, columns=columns)
    
    return df

def calculate_improvement(baseline_results, finetuned_results):
    """
    파인튜닝으로 인한 성능 개선 계산
    
    Returns:
        개선율 딕셔너리
    """
    if not baseline_results or not finetuned_results:
        return None
    
    metrics = ['rouge1', 'rouge2', 'rougeL', 'bleu', 'cosine_similarity']
    improvements = {}
    
    for metric in metrics:
        baseline_score = baseline_results[metric]
        finetuned_score = finetuned_results[metric]
        
        if baseline_score > 0:
            improvement = ((finetuned_score - baseline_score) / baseline_score) * 100
            improvements[metric] = improvement
        else:
            improvements[metric] = 0
    
    return improvements

# 결과 분석 및 표시
if baseline_results:
    print("\n" + "=" * 80)
    print("📊 모델 성능 비교 결과")
    print("=" * 80)
    
    # 비교 테이블 생성
    comparison_df = create_comparison_table(baseline_results, finetuned_results)
    
    # 결과 테이블 출력
    print("\n📋 전체 성능 비교:")
    print(comparison_df.round(4).to_string(index=False))
    
    # 개선율 계산 및 출력
    if finetuned_results:
        improvements = calculate_improvement(baseline_results, finetuned_results)
        
        if improvements:
            print("\n🚀 파인튜닝으로 인한 성능 개선:")
            print("-" * 50)
            
            improvement_data = []
            for metric, improvement in improvements.items():
                metric_name = {
                    'rouge1': 'ROUGE-1',
                    'rouge2': 'ROUGE-2', 
                    'rougeL': 'ROUGE-L',
                    'bleu': 'BLEU',
                    'cosine_similarity': '코사인 유사도'
                }[metric]
                
                if improvement > 0:
                    status = "📈 개선"
                elif improvement < 0:
                    status = "📉 악화"
                else:
                    status = "➖ 동일"
                
                improvement_data.append([metric_name, f"{improvement:+.2f}%", status])
                print(f"  {metric_name}: {improvement:+.2f}% {status}")
            
            # 종합 개선 평가
            avg_improvement = np.mean(list(improvements.values()))
            print(f"\n🎯 종합 평균 개선율: {avg_improvement:+.2f}%")
            
            if avg_improvement > 5:
                print("🌟 파인튜닝이 상당한 성능 향상을 가져왔습니다!")
            elif avg_improvement > 1:
                print("✅ 파인튜닝이 성능을 개선했습니다.")
            elif avg_improvement > -1:
                print("➖ 파인튜닝 효과가 미미합니다.")
            else:
                print("⚠️ 파인튜닝 후 성능이 다소 저하되었습니다.")
    
    else:
        print("\n⚠️ 파인튜닝 모델이 없어 성능 비교를 수행할 수 없습니다.")
        print("💡 먼저 03_fine_tuning_with_lora.ipynb를 완료하세요.")
        
else:
    print("❌ 평가 결과가 없어 성능 비교를 수행할 수 없습니다.")

## 6. 성능 비교 시각화

### 📊 베이스라인 vs 파인튜닝 모델 성능 차이를 시각적으로 분석
- **막대 차트**: 각 메트릭별 직접적인 성능 비교  
- **방사형 차트**: 종합적인 성능 프로필 비교
- **개선율 차트**: 파인튜닝으로 인한 개선 정도 시각화
- **응답 예시 비교**: 실제 생성 결과 질적 분석

In [ ]:
def create_performance_visualization(baseline_results, finetuned_results=None):
    """
    베이스라인 vs 파인튜닝 모델 성능 비교 시각화 생성
    
    이 함수는 여러 관점에서 모델 성능을 시각적으로 비교합니다:
    1. 📊 메트릭별 막대 차트: 각 성능 지표의 직접적인 비교
    2. 🎯 방사형 차트: 전체적인 성능 프로필 비교  
    3. 📈 개선율 차트: 파인튜닝으로 인한 개선 정도
    4. ⏱️ 응답 시간 비교: 실용성 측면의 성능 분석
    """
    
    if not baseline_results:
        print("❌ 베이스라인 결과가 없어 시각화를 생성할 수 없습니다.")
        return None
    
    print("🎨 성능 비교 시각화 생성 중...")
    
    # 시각화 설정
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('🎯 베이스라인 vs 파인튜닝 모델 종합 성능 비교', 
                 fontsize=16, fontweight='bold', y=0.98)
    
    # 1. 메트릭별 막대 차트 (좌상단)
    # 📊 의미: 각 평가 지표에서 두 모델의 성능을 직접적으로 비교
    # - ROUGE: 텍스트 중복도 기반 유사성 (높을수록 좋음)
    # - BLEU: 번역 품질 측정 지표 (높을수록 좋음) 
    # - 코사인 유사도: 의미적 유사성 (높을수록 좋음)
    metrics = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BLEU', '코사인 유사도']
    baseline_scores = [
        baseline_results['rouge1'], baseline_results['rouge2'], 
        baseline_results['rougeL'], baseline_results['bleu'], 
        baseline_results['cosine_similarity']
    ]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    bars1 = axes[0, 0].bar(x - width/2, baseline_scores, width, 
                          label='🤖 베이스라인', color='lightblue', alpha=0.8)
    
    if finetuned_results:
        finetuned_scores = [
            finetuned_results['rouge1'], finetuned_results['rouge2'],
            finetuned_results['rougeL'], finetuned_results['bleu'],
            finetuned_results['cosine_similarity']
        ]
        bars2 = axes[0, 0].bar(x + width/2, finetuned_scores, width,
                              label='🎯 파인튜닝', color='lightcoral', alpha=0.8)
        
        # 개선된 메트릭에 별표 표시
        for i, (baseline, finetuned) in enumerate(zip(baseline_scores, finetuned_scores)):
            if finetuned > baseline:
                axes[0, 0].text(i, max(baseline, finetuned) + 0.01, '⭐', 
                               ha='center', va='bottom', fontsize=12)
    
    axes[0, 0].set_xlabel('평가 메트릭')
    axes[0, 0].set_ylabel('점수')
    axes[0, 0].set_title('📊 메트릭별 성능 비교\n(⭐: 파인튜닝이 더 우수)')
    axes[0, 0].set_xticks(x)
    axes[0, 0].set_xticklabels(metrics, rotation=45, ha='right')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 방사형 차트 (우상단) - 종합적인 성능 프로필 비교
    # 📊 의미: 다각형 모양으로 모델의 전체적인 강약점을 한눈에 파악
    # - 면적이 클수록 전반적인 성능이 우수
    # - 각 꼭짓점은 특정 성능 지표를 나타냄
    if finetuned_results:
        angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
        angles += angles[:1]  # 닫힌 다각형을 위해
        
        baseline_scores_radar = baseline_scores + baseline_scores[:1]
        finetuned_scores_radar = finetuned_scores + finetuned_scores[:1]
        
        ax_radar = plt.subplot(2, 2, 2, projection='polar')
        ax_radar.plot(angles, baseline_scores_radar, 'o-', linewidth=2, 
                     label='🤖 베이스라인', color='blue', alpha=0.7)
        ax_radar.fill(angles, baseline_scores_radar, alpha=0.25, color='blue')
        
        ax_radar.plot(angles, finetuned_scores_radar, 'o-', linewidth=2,
                     label='🎯 파인튜닝', color='red', alpha=0.7)
        ax_radar.fill(angles, finetuned_scores_radar, alpha=0.25, color='red')
        
        ax_radar.set_xticks(angles[:-1])
        ax_radar.set_xticklabels(metrics)
        ax_radar.set_ylim(0, 1)
        ax_radar.set_title('🎯 종합 성능 프로필 비교\n(면적이 클수록 우수)')
        ax_radar.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
        ax_radar.grid(True)
    else:
        axes[0, 1].text(0.5, 0.5, '⚠️ 파인튜닝 모델이 없어\n방사형 차트를 생성할 수 없습니다.\n\n💡 03번 노트북을 먼저 완료하세요.',
                       ha='center', va='center', fontsize=12, 
                       bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow"))
        axes[0, 1].set_title('🎯 종합 성능 프로필 비교')
        axes[0, 1].axis('off')
    
    # 3. 개선율 차트 (좌하단)
    # 📊 의미: 파인튜닝으로 인한 성능 변화를 백분율로 표시
    # - 양수: 성능 향상, 음수: 성능 저하
    # - 막대의 색깔로 개선/저하를 직관적으로 표시
    if finetuned_results:
        improvements = []
        for baseline, finetuned in zip(baseline_scores, finetuned_scores):
            if baseline > 0:
                improvement = ((finetuned - baseline) / baseline) * 100
                improvements.append(improvement)
            else:
                improvements.append(0)
        
        colors = ['green' if imp > 0 else 'red' if imp < 0 else 'gray' for imp in improvements]
        bars = axes[1, 0].bar(metrics, improvements, color=colors, alpha=0.7)
        
        # 개선율 수치 표시
        for bar, improvement in zip(bars, improvements):
            height = bar.get_height()
            axes[1, 0].text(bar.get_x() + bar.get_width()/2., height + (1 if height > 0 else -3),
                           f'{improvement:+.1f}%', ha='center', va='bottom' if height > 0 else 'top',
                           fontweight='bold')
        
        axes[1, 0].axhline(y=0, color='black', linestyle='-', alpha=0.3)
        axes[1, 0].set_xlabel('평가 메트릭')
        axes[1, 0].set_ylabel('개선율 (%)')
        axes[1, 0].set_title('📈 파인튜닝 개선율\n(초록: 개선, 빨강: 저하)')
        axes[1, 0].tick_params(axis='x', rotation=45)
        axes[1, 0].grid(True, alpha=0.3)
        
        # 평균 개선율 표시
        avg_improvement = np.mean(improvements)
        axes[1, 0].text(0.02, 0.98, f'평균 개선율: {avg_improvement:+.1f}%', 
                       transform=axes[1, 0].transAxes, fontsize=12, fontweight='bold',
                       bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue" if avg_improvement > 0 else "lightcoral"))
    else:
        axes[1, 0].text(0.5, 0.5, '⚠️ 파인튜닝 모델이 없어\n개선율을 계산할 수 없습니다.\n\n💡 03번 노트북을 먼저 완료하세요.',
                       ha='center', va='center', fontsize=12,
                       bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow"))
        axes[1, 0].set_title('📈 파인튜닝 개선율')
        axes[1, 0].axis('off')
    
    # 4. 응답 시간 및 길이 비교 (우하단)  
    # 📊 의미: 모델의 실용성 측면 비교
    # - 응답 시간: 실제 서비스에서의 사용성 (짧을수록 좋음)
    # - 응답 길이: 답변의 상세함 정도
    response_metrics = ['평균 응답 시간(초)', '평균 응답 길이(단어)']
    baseline_practical = [baseline_results['avg_generation_time'], baseline_results['avg_prediction_length']]
    
    if finetuned_results:
        finetuned_practical = [finetuned_results['avg_generation_time'], finetuned_results['avg_prediction_length']]
        
        x = np.arange(len(response_metrics))
        bars1 = axes[1, 1].bar(x - width/2, baseline_practical, width,
                              label='🤖 베이스라인', color='lightblue', alpha=0.8)
        bars2 = axes[1, 1].bar(x + width/2, finetuned_practical, width,
                              label='🎯 파인튜닝', color='lightcoral', alpha=0.8)
        
        # 수치 표시
        for bars in [bars1, bars2]:
            for bar in bars:
                height = bar.get_height()
                axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                               f'{height:.2f}', ha='center', va='bottom', fontsize=10)
        
        axes[1, 1].legend()
    else:
        bars = axes[1, 1].bar(response_metrics, baseline_practical, 
                             color='lightblue', alpha=0.8, label='🤖 베이스라인')
        for bar in bars:
            height = bar.get_height()
            axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                           f'{height:.2f}', ha='center', va='bottom', fontsize=10)
        axes[1, 1].legend()
    
    axes[1, 1].set_xlabel('실용성 지표')
    axes[1, 1].set_ylabel('값')
    axes[1, 1].set_title('⏱️ 실용성 비교\n(응답 속도 및 길이)')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.93)  # 제목 여백 조정
    
    # 그래프 저장
    plt.savefig('model_performance_comparison.png', dpi=300, bbox_inches='tight', 
                facecolor='white', edgecolor='none')
    plt.show()
    
    print("✅ 성능 비교 시각화 완료")
    print("💾 저장된 파일: model_performance_comparison.png")
    
    # 시각화 해석 가이드 출력
    print("\n📖 시각화 해석 가이드:")
    print("  📊 메트릭별 비교: 각 지표에서 어느 모델이 우수한지 직접 비교")
    print("  🎯 방사형 차트: 모델의 전체적인 성능 밸런스를 면적으로 비교")  
    print("  📈 개선율 차트: 파인튜닝으로 인한 구체적인 개선 정도 (% 단위)")
    print("  ⏱️ 실용성 비교: 실제 사용 시 고려해야 할 속도와 응답 품질")
    
    return fig

# 성능 비교 시각화 실행
if baseline_results:
    comparison_fig = create_performance_visualization(baseline_results, finetuned_results)
else:
    print("❌ 베이스라인 결과가 없어 시각화를 생성할 수 없습니다.")

In [ ]:
def show_response_examples(baseline_results, finetuned_results=None, num_examples=3):
    """
    베이스라인과 파인튜닝 모델의 실제 응답 예시를 비교하여 출력
    
    이 함수는 정량적 메트릭으로는 파악하기 어려운 질적 차이를 보여줍니다:
    - 답변의 정확성: 컨텍스트를 얼마나 정확히 활용했는가
    - 답변의 완성도: 질문에 대한 충분한 답변을 제공했는가  
    - 답변의 일관성: 주어진 정보에 기반한 논리적 답변인가
    - 언어 품질: 자연스럽고 읽기 쉬운 한국어인가
    """
    
    print("\n" + "=" * 100)
    print("🔍 베이스라인 vs 파인튜닝 모델 응답 예시 비교")
    print("=" * 100)
    print("📋 이 비교를 통해 정량적 메트릭으로는 보기 어려운 질적 차이를 확인할 수 있습니다.")
    print("   - 답변의 정확성과 완성도")
    print("   - 컨텍스트 활용 능력") 
    print("   - 한국어 답변의 자연스러움")
    print("=" * 100)
    
    if not baseline_results:
        print("❌ 베이스라인 결과가 없어 예시를 출력할 수 없습니다.")
        return
    
    # 비교할 예시 개수 결정
    max_examples = min(num_examples, len(baseline_results['predictions']))
    
    for i in range(max_examples):
        print(f"\n【예시 {i+1}】")
        print("─" * 80)
        
        # 컨텍스트 및 질문 출력
        context = baseline_results['contexts'][i]
        question = baseline_results['questions'][i]
        reference = baseline_results['references'][i]
        
        print(f"🌐 컨텍스트:")
        print(f"   {context[:200]}{'...' if len(context) > 200 else ''}")
        print(f"\n❓ 질문:")
        print(f"   {question}")
        print(f"\n🎯 정답:")
        print(f"   {reference}")
        
        print(f"\n💬 모델 응답 비교:")
        print("─" * 50)
        
        # 베이스라인 응답
        baseline_pred = baseline_results['predictions'][i]
        print(f"🤖 베이스라인 모델:")
        print(f"   {baseline_pred}")
        
        # 파인튜닝 응답 (있는 경우)
        if finetuned_results and i < len(finetuned_results['predictions']):
            finetuned_pred = finetuned_results['predictions'][i]
            print(f"\n🎯 파인튜닝 모델:")
            print(f"   {finetuned_pred}")
            
            # 간단한 질적 비교 분석
            print(f"\n📊 간단 분석:")
            
            # 길이 비교
            baseline_len = len(baseline_pred.split())
            finetuned_len = len(finetuned_pred.split())
            print(f"   길이: 베이스라인 {baseline_len}단어 vs 파인튜닝 {finetuned_len}단어")
            
            # 정답과의 유사성 (간단한 키워드 매칭)
            ref_keywords = set(reference.lower().split())
            baseline_keywords = set(baseline_pred.lower().split())
            finetuned_keywords = set(finetuned_pred.lower().split())
            
            baseline_overlap = len(ref_keywords & baseline_keywords) / len(ref_keywords) if ref_keywords else 0
            finetuned_overlap = len(ref_keywords & finetuned_keywords) / len(ref_keywords) if ref_keywords else 0
            
            print(f"   정답 키워드 매칭: 베이스라인 {baseline_overlap:.1%} vs 파인튜닝 {finetuned_overlap:.1%}")
            
            # 어느 모델이 더 나은지 간단한 판단
            if finetuned_overlap > baseline_overlap + 0.1:  # 10% 이상 차이
                print(f"   🌟 파인튜닝 모델이 더 정확한 답변을 생성했습니다.")
            elif baseline_overlap > finetuned_overlap + 0.1:
                print(f"   🔄 베이스라인 모델이 더 정확한 답변을 생성했습니다.")
            else:
                print(f"   ➖ 두 모델의 답변 품질이 비슷합니다.")
            
        else:
            print(f"\n⚠️ 파인튜닝 모델 응답이 없습니다.")
        
        print("\n" + "─" * 80)
        
        # 예시 간 구분선
        if i < max_examples - 1:
            print("\n")
    
    # 전체적인 질적 평가 요약
    if finetuned_results:
        print(f"\n📋 전체 응답 품질 비교 요약:")
        print("─" * 50)
        
        # 전체 응답에 대한 간단한 통계
        total_baseline_length = np.mean([len(pred.split()) for pred in baseline_results['predictions']])
        total_finetuned_length = np.mean([len(pred.split()) for pred in finetuned_results['predictions']])
        
        print(f"평균 응답 길이: 베이스라인 {total_baseline_length:.1f}단어 vs 파인튜닝 {total_finetuned_length:.1f}단어")
        
        # 전반적인 품질 평가 (ROUGE-L 기준)
        baseline_quality = baseline_results['rougeL']
        finetuned_quality = finetuned_results['rougeL']
        
        quality_improvement = ((finetuned_quality - baseline_quality) / baseline_quality) * 100 if baseline_quality > 0 else 0
        
        if quality_improvement > 5:
            overall_assessment = "🌟 파인튜닝으로 답변 품질이 크게 향상되었습니다!"
        elif quality_improvement > 1:
            overall_assessment = "✅ 파인튜닝으로 답변 품질이 개선되었습니다."
        elif quality_improvement > -1:
            overall_assessment = "➖ 두 모델의 답변 품질이 비슷합니다."
        else:
            overall_assessment = "🔄 베이스라인 모델의 답변이 더 나은 경우도 있습니다."
        
        print(f"\n전반적 평가: {overall_assessment}")
        print(f"품질 개선율: {quality_improvement:+.1f}% (ROUGE-L 기준)")
    
    else:
        print(f"\n⚠️ 파인튜닝 모델이 없어 질적 비교를 수행할 수 없습니다.")
        print(f"💡 먼저 03_fine_tuning_with_lora.ipynb를 완료하여 파인튜닝된 모델을 생성하세요.")

# 응답 예시 비교 실행
if baseline_results:
    show_response_examples(baseline_results, finetuned_results, num_examples=3)
else:
    print("❌ 베이스라인 결과가 없어 응답 예시를 출력할 수 없습니다.")

In [ ]:
# 평가 결과 저장 및 요약
def save_evaluation_results(baseline_results, finetuned_results=None):
    """
    평가 결과를 JSON 파일로 저장하고 최종 요약을 생성
    """
    print("💾 평가 결과 저장 중...")
    
    # 결과 저장용 딕셔너리 생성
    evaluation_summary = {
        'evaluation_date': pd.Timestamp.now().isoformat(),
        'baseline_model': {
            'name': baseline_results['model_name'],
            'sample_count': baseline_results['sample_count'],
            'rouge1': float(baseline_results['rouge1']),
            'rouge2': float(baseline_results['rouge2']),
            'rougeL': float(baseline_results['rougeL']),
            'bleu': float(baseline_results['bleu']),
            'cosine_similarity': float(baseline_results['cosine_similarity']),
            'avg_generation_time': float(baseline_results['avg_generation_time']),
            'avg_prediction_length': float(baseline_results['avg_prediction_length'])
        }
    }
    
    # 파인튜닝 결과가 있는 경우 추가
    if finetuned_results:
        evaluation_summary['finetuned_model'] = {
            'name': finetuned_results['model_name'],
            'sample_count': finetuned_results['sample_count'],
            'rouge1': float(finetuned_results['rouge1']),
            'rouge2': float(finetuned_results['rouge2']),
            'rougeL': float(finetuned_results['rougeL']),
            'bleu': float(finetuned_results['bleu']),
            'cosine_similarity': float(finetuned_results['cosine_similarity']),
            'avg_generation_time': float(finetuned_results['avg_generation_time']),
            'avg_prediction_length': float(finetuned_results['avg_prediction_length'])
        }
        
        # 개선율 계산 및 저장
        improvements = calculate_improvement(baseline_results, finetuned_results)
        if improvements:
            evaluation_summary['improvement_analysis'] = {
                'rouge1_improvement': float(improvements['rouge1']),
                'rouge2_improvement': float(improvements['rouge2']),
                'rougeL_improvement': float(improvements['rougeL']),
                'bleu_improvement': float(improvements['bleu']),
                'cosine_similarity_improvement': float(improvements['cosine_similarity']),
                'average_improvement': float(np.mean(list(improvements.values())))
            }
    
    # JSON 파일로 저장
    with open('baseline_vs_finetuned_evaluation.json', 'w', encoding='utf-8') as f:
        json.dump(evaluation_summary, f, ensure_ascii=False, indent=2)
    
    print("✅ 평가 결과 저장 완료")
    print("📁 저장된 파일:")
    print("  - baseline_vs_finetuned_evaluation.json: 종합 평가 결과")
    print("  - model_performance_comparison.png: 성능 비교 시각화")
    
    return evaluation_summary

# 평가 결과 저장 실행
if baseline_results:
    evaluation_summary = save_evaluation_results(baseline_results, finetuned_results)
else:
    print("❌ 저장할 평가 결과가 없습니다.")

## 9. 📋 Day 1 실습 최종 요약 및 결론

### ✅ 완료된 작업
1. **데이터 전처리 (01번)**: RAFT 방법론을 활용한 한국어 RAG 데이터셋 생성
2. **데이터 품질 검증 (02번)**: 토큰 분포, 중복성, RAFT 구조 검증
3. **QLoRA 파인튜닝 (03번)**: 4-bit 양자화를 활용한 효율적 EXAONE 모델 파인튜닝
4. **성능 평가 및 비교 (04번)**: 베이스라인 vs 파인튜닝 모델 정량적/정성적 비교

### 🎯 핵심 성과
- **RAFT 기반 파인튜닝**: RAG 성능 향상에 특화된 데이터 전처리 및 학습
- **메모리 효율적 학습**: QLoRA를 통한 Colab 무료 환경에서의 대규모 모델 파인튜닝
- **종합적 평가**: ROUGE, BLEU, 코사인 유사도 등 다각도 성능 측정
- **실용적 검증**: 실제 RAG 시나리오에서의 성능 개선 확인

### 📊 기대 효과
파인튜닝을 통해 다음과 같은 개선을 기대할 수 있습니다:
- **정확성 향상**: 주어진 컨텍스트를 더 정확히 활용한 답변 생성
- **일관성 개선**: RAFT 훈련을 통한 컨텍스트 기반 추론 능력 강화
- **한국어 품질**: 한국어 특화 데이터셋으로 인한 자연스러운 답변 생성

### 💡 실무 활용 방안
1. **기업 내 RAG 시스템**: 사내 문서 기반 질의응답 시스템 구축
2. **고객 서비스 봇**: FAQ 기반 자동 응답 시스템의 정확도 향상
3. **교육 플랫폼**: 교재 내용 기반 학습 도우미 개발
4. **연구 지원 도구**: 논문/자료 기반 연구 질의응답 시스템

### 🔄 추가 개선 방향
1. **더 많은 데이터**: 도메인별 특화 데이터셋으로 성능 향상
2. **하이퍼파라미터 튜닝**: Learning rate, LoRA rank 등 최적화
3. **앙상블 기법**: 여러 파인튜닝 모델의 결합으로 성능 향상
4. **지속적 학습**: 새로운 데이터를 통한 모델 업데이트

In [ ]:
# 🎯 Day 1 실습 4 최종 완료!
print("🎉 Day 1 실습 4: 베이스라인 vs 파인튜닝 모델 성능 비교 완료!")
print("=" * 80)

# 최종 결과 요약 출력
if baseline_results:
    print("📊 평가 완료된 모델:")
    print(f"  ✅ 베이스라인 모델: {baseline_results['model_name']}")
    if finetuned_results:
        print(f"  ✅ 파인튜닝 모델: {finetuned_results['model_name']}")
        print("\n🔥 베이스라인 vs 파인튜닝 완전 비교 성공!")
    else:
        print("  ⚠️ 파인튜닝 모델: 비교 불가")
        print("\n💡 파인튜닝 모델이 로드되지 않은 이유:")
        print("   1. 03번 노트북의 파인튜닝이 완료되지 않았음")
        print("   2. 모델 저장 과정에서 오류 발생")
        print("   3. 파일 경로나 권한 문제")
        print("\n🔧 해결 방법:")
        print("   👉 03_fine_tuning_with_lora.ipynb를 처음부터 끝까지 완료")
        print("   👉 특히 23번 셀(모델 저장)에서 '✅ 모델 저장 성공!' 메시지 확인")
        print("   👉 이 노트북을 다시 실행하여 완전한 비교 수행")
    
    print(f"\n📈 평가 샘플 수: {baseline_results['sample_count']}개")
    
    # 핵심 메트릭 요약
    if finetuned_results:
        print("\n🎯 핵심 성능 지표:")
        metrics = ['rouge1', 'rougeL', 'bleu', 'cosine_similarity']
        metric_names = ['ROUGE-1', 'ROUGE-L', 'BLEU', '코사인 유사도']
        
        for metric, name in zip(metrics, metric_names):
            baseline_score = baseline_results[metric]
            finetuned_score = finetuned_results[metric]
            improvement = ((finetuned_score - baseline_score) / baseline_score * 100) if baseline_score > 0 else 0
            
            if improvement > 1:
                status = "📈"
            elif improvement < -1:
                status = "📉"
            else:
                status = "➖"
            
            print(f"  {name}: {baseline_score:.3f} → {finetuned_score:.3f} ({improvement:+.1f}%) {status}")
        
        # 전체 개선율
        improvements = calculate_improvement(baseline_results, finetuned_results)
        if improvements:
            avg_improvement = np.mean(list(improvements.values()))
            print(f"\n🌟 평균 성능 개선: {avg_improvement:+.1f}%")
            
            if avg_improvement > 5:
                print("🚀 파인튜닝이 상당한 성능 향상을 달성했습니다!")
            elif avg_improvement > 1:
                print("✅ 파인튜닝으로 성능이 개선되었습니다.")
            else:
                print("💡 파인튜닝 효과를 더 높이기 위한 추가 최적화를 고려해보세요.")
    
    else:
        print("\n📊 베이스라인 모델 성능 (단독 평가):")
        print(f"  ROUGE-1: {baseline_results['rouge1']:.3f}")
        print(f"  ROUGE-L: {baseline_results['rougeL']:.3f}")
        print(f"  BLEU: {baseline_results['bleu']:.3f}")
        print(f"  코사인 유사도: {baseline_results['cosine_similarity']:.3f}")
        print(f"\n📝 베이스라인 결과 해석:")
        print(f"  - 베이스라인 모델의 RAG 성능을 측정했습니다")
        print(f"  - 파인튜닝 모델과 비교하면 개선 효과를 확인할 수 있습니다")
        print(f"  - 현재 결과는 향후 파인튜닝 효과 측정의 기준점이 됩니다")
    
    print(f"\n📁 생성된 결과 파일:")
    print(f"  - baseline_vs_finetuned_evaluation.json (평가 결과)")
    print(f"  - model_performance_comparison.png (시각화)")

else:
    print("❌ 평가 결과가 없습니다.")
    print("💡 베이스라인 모델도 로드되지 않았다면:")
    print("   1. GPU 메모리 부족일 수 있습니다 → 런타임 재시작")
    print("   2. 라이브러리 설치 문제일 수 있습니다 → 1-2번 셀 재실행") 
    print("   3. 평가 데이터가 없을 수 있습니다 → 01번 노트북 먼저 실행")

print(f"\n🎓 Day 1 실습 시리즈 완료!")
print("=" * 80)
print("✅ 01번: RAFT 데이터 전처리 및 검증")
print("✅ 02번: 데이터 품질 분석 및 시각화") 
print("✅ 03번: QLoRA 파인튜닝 실행")
print("✅ 04번: 베이스라인 vs 파인튜닝 성능 비교")
print("")

# 실습 완료 수준 확인
if 'finetuned_results' in locals() and finetuned_results:
    completion_level = "🌟 완전 완료"
    print(f"{completion_level}: 모든 실습이 성공적으로 완료되었습니다!")
    print("🏆 베이스라인 vs 파인튜닝 모델의 성능 비교까지 모든 단계를 마쳤습니다!")
elif 'baseline_results' in locals() and baseline_results:
    completion_level = "⚠️ 부분 완료"
    print(f"{completion_level}: 베이스라인 평가까지는 완료되었습니다.")
    print("🎯 완전한 실습 완료를 위해서는:")
    print("   1. 03_fine_tuning_with_lora.ipynb 완료")
    print("   2. 이 노트북 재실행으로 파인튜닝 모델 비교")
else:
    completion_level = "🔄 재실행 필요"
    print(f"{completion_level}: 실습 환경에 문제가 있었습니다.")
    print("💡 해결책:")
    print("   1. 런타임 재시작 후 처음부터 재실행")
    print("   2. 01번 → 02번 → 03번 → 04번 순서로 완료")

print("")
print("🚀 이제 여러분은 다음 능력을 갖추었습니다:")
print("   • RAG 성능 향상을 위한 RAFT 데이터 전처리")
print("   • QLoRA를 활용한 효율적 대규모 모델 파인튜닝")
print("   • 다양한 메트릭을 활용한 종합적 모델 성능 평가")
print("   • 베이스라인과 파인튜닝 모델의 정량적/정성적 비교")
print("")
print("💡 다음 단계 제안:")
if 'finetuned_results' in locals() and finetuned_results:
    print("   🎉 축하합니다! 모든 실습이 완료되었습니다.")
    print("   • 실제 프로덕션 환경에 모델 적용 고려")
    print("   • 도메인별 특화 데이터로 추가 파인튜닝 시도")
    print("   • 더 큰 모델이나 다른 파인튜닝 기법 실험")
    print("   • 허깅페이스 Hub에 모델 업로드 및 공유")
else:
    print("   🔄 03번 노트북을 완료하여 파인튜닝 모델 생성")
    print("   🔁 이 노트북을 다시 실행하여 완전한 성능 비교")
    print("   📈 파인튜닝 효과를 정량적으로 측정")
    print("   🎯 베이스라인 대비 성능 개선 확인")

print(f"\n🌟 RAFT 방법론을 활용한 한국어 RAG 모델 파인튜닝 실습")
print(f"   완료 수준: {completion_level}")
print("🎓 한국어 RAG 시스템 구축 역량을 획득하셨습니다!")